In [20]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.


In [14]:
# Load cifar10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [15]:
# Check the shape of the training and test data
print("Train Images Shape:", train_images.shape)
print("Test Images Shape:", test_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Labels Shape:", test_labels.shape)

Train Images Shape: (50000, 32, 32, 3)
Test Images Shape: (10000, 32, 32, 3)
Train Labels Shape: (50000, 1)
Test Labels Shape: (10000, 1)


In [16]:
train_images = train_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0

In [17]:
train_images.shape

(50000, 32, 32, 3)

In [18]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),)(
        inputs
    )
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),)(
        x
    )
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(
        128, 3, padding="same", kernel_regularizer=regularizers.l2(0.01),
    )(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01),)(
        x
    )
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [21]:
model = my_model()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 64)        18496 

In [22]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [23]:
model.fit (train_images, train_labels, batch_size=64, epochs=10, verbose =2 )

Epoch 1/10
782/782 - 9s - loss: 3.0372 - accuracy: 0.3454 - 9s/epoch - 11ms/step
Epoch 2/10
782/782 - 4s - loss: 1.9301 - accuracy: 0.4551 - 4s/epoch - 5ms/step
Epoch 3/10
782/782 - 4s - loss: 1.6425 - accuracy: 0.5040 - 4s/epoch - 6ms/step
Epoch 4/10
782/782 - 4s - loss: 1.5135 - accuracy: 0.5392 - 4s/epoch - 5ms/step
Epoch 5/10
782/782 - 4s - loss: 1.4427 - accuracy: 0.5601 - 4s/epoch - 5ms/step
Epoch 6/10
782/782 - 4s - loss: 1.3802 - accuracy: 0.5834 - 4s/epoch - 5ms/step
Epoch 7/10
782/782 - 4s - loss: 1.3518 - accuracy: 0.5934 - 4s/epoch - 5ms/step
Epoch 8/10
782/782 - 4s - loss: 1.3249 - accuracy: 0.6024 - 4s/epoch - 5ms/step
Epoch 9/10
782/782 - 4s - loss: 1.2988 - accuracy: 0.6150 - 4s/epoch - 5ms/step
Epoch 10/10
782/782 - 4s - loss: 1.2799 - accuracy: 0.6214 - 4s/epoch - 6ms/step


In [24]:
model.evaluate(test_images, test_labels, batch_size=64, verbose =2 )

157/157 - 1s - loss: 1.3489 - accuracy: 0.5971 - 583ms/epoch - 4ms/step


[1.3488680124282837, 0.597100019454956]